In [ ]:
!pip install pandas numpy statsmodels openpyxl

In [ ]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [ ]:
# -----------------------------
# Paths (match your Stata globals)
# -----------------------------
path = "/Users/gaoyujuan/REAP Dropbox/Gao yujuan/lightning_bolts"
productdir = f"{path}/Interim_Data_Product/"
dhsdir = f"{path}/processed/dhs/"
dofiledir = f"{path}/scripts/programs for analysis/"
resultdir = f"{path}/Tables/"
tempdir = f"{path}/processed/temp/"
finaldir = f"{path}/Final Results/"

os.makedirs(resultdir, exist_ok=True)
os.makedirs(tempdir, exist_ok=True)
os.makedirs(finaldir, exist_ok=True)


In [ ]:
# -----------------------------
# Helpers
# -----------------------------
def zscore(s):
    s = pd.to_numeric(s, errors="coerce")
    return (s - s.mean()) / s.std(ddof=0)

def fe_dummies(df, fe_spec):
    """
    fe_spec: "lga DHSYEAR" or "lga" or "lga + DHSYEAR"
    Returns a DataFrame of FE dummies (without the first level per group).
    """
    if not fe_spec:
        return pd.DataFrame(index=df.index)
    parts = [p for p in fe_spec.replace("+"," ").split() if p.strip()]
    # combine all dummies together
    mats = []
    for p in parts:
        if p not in df.columns:
            continue
        d = pd.get_dummies(df[p], prefix=p, drop_first=True)
        mats.append(d)
    if len(mats)==0:
        return pd.DataFrame(index=df.index)
    return pd.concat(mats, axis=1)

def wls_clustered(y, X, w, cluster):
    """
    Weighted OLS with cluster-robust SEs by 'cluster' (Series).
    Returns fitted result.
    """
    w = pd.to_numeric(w, errors="coerce").fillna(0.0)
    model = sm.WLS(y, sm.add_constant(X, has_constant='add'), weights=w)
    if cluster is not None:
        res = model.fit(cov_type='cluster', cov_kwds={'groups': cluster})
    else:
        res = model.fit()
    return res

def control_mean(df, outcome, ctrl_var_name):
    """Mean of outcome where the *unstandardized* control var equals 0 and observation in sample."""
    if ctrl_var_name not in df.columns:
        return np.nan
    sub = df.loc[df[ctrl_var_name]==0, outcome]
    return float(sub.mean()) if len(sub)>0 else np.nan

def run_block(
    df,
    outcomes,
    bfd_list,
    std_prefix,          # e.g., "std_lt_GSMCOVER_dist{bfd}"
    raw_prefix,          # e.g., "lt_GSMCOVER_dist{bfd}"
    file_suffix,         # e.g., "employment_{label}_lga.xlsx" (we will write all labels to one workbook later)
    title_prefix,        # e.g., "Impact of Mobile Internet Coverage on Employment - {label}"
    controls_weather,    # list
    controls_individual, # list (may include categorical that are already coded)
    condition_pairs,     # list of tuples: (condition_name, condition_label)
    fixed_effect_logic   # callable(condition_name)->fe_spec string
):
    """
    For each condition and each bfd, run WLS with FE dummies and cluster-robust SEs.
    Returns dict: {label: DataFrame of results}
    """
    results_per_label = {}
    for idx, (cond, label) in enumerate(condition_pairs, 1):
        if cond == "full":
            sub = df.copy()
        else:
            # Evaluate condition in pandas .query() where possible; for safety, use eval on a boolean mask
            try:
                sub = df.query(cond).copy()
            except Exception:
                # fallback to python eval row-wise — slower but resilient
                mask = df.eval(cond)
                sub = df.loc[mask].copy()

        # choose FE
        fe_spec = fixed_effect_logic(cond)
        fe_mat = fe_dummies(sub, fe_spec)
        cluster = sub["cluster_id"] if "cluster_id" in sub.columns else None

        # columns to include
        base_vars = []
        for lst in (controls_weather, controls_individual):
            for v in lst:
                if v in sub.columns:
                    base_vars.append(v)

        # coerce categorical controls if like i.highest_education -> already numeric in data
        # Build results rows
        out_rows = []

        for bfd in bfd_list:
            std_var = std_prefix.format(bfd=bfd)
            raw_var = raw_prefix.format(bfd=bfd)

            # skip if std var missing
            if std_var not in sub.columns:
                continue

            X_base = sub[base_vars].copy()
            X = pd.concat([X_base, fe_mat, sub[[std_var]]], axis=1)

            for outcome in outcomes:
                if outcome not in sub.columns:
                    continue
                y = sub[outcome]
                w = sub["wgt"] if "wgt" in sub.columns else pd.Series(np.ones(len(sub)), index=sub.index)
                res = wls_clustered(y, X, w, cluster)
                # extract coef & se for std_var
                beta = res.params.get(std_var, np.nan)
                se = res.bse.get(std_var, np.nan)
                pval = res.pvalues.get(std_var, np.nan)
                cm = control_mean(sub, outcome, raw_var)

                out_rows.append({
                    "label": label,
                    "condition": cond,
                    "fixed_effects": fe_spec,
                    "outcome": outcome,
                    "bfd": bfd,
                    "coef": beta,
                    "se": se,
                    "pval": pval,
                    "N": int(res.nobs),
                    "control_mean_raw0": cm,
                    "title": f"{title_prefix.format(label=label)}"
                })

        results_per_label[label] = pd.DataFrame(out_rows)

    return results_per_label

def write_results_dict_to_workbook(results_dict, out_path):
    """results_dict: {label: DataFrame}; write each label as a sheet."""
    with pd.ExcelWriter(out_path, engine="openpyxl") as xw:
        for label, df_sheet in results_dict.items():
            # order columns nicely
            cols = ["title","label","condition","fixed_effects","outcome","bfd","coef","se","pval","N","control_mean_raw0"]
            use_cols = [c for c in cols if c in df_sheet.columns] + [c for c in df_sheet.columns if c not in cols]
            df_sheet[use_cols].to_excel(xw, sheet_name=label[:31], index=False)

In [ ]:
# ============================================================
# Section 1: Employment (contemporaneous) — outcomes set 1
# ============================================================
# Load contemporaneous
contemp = pd.read_stata(os.path.join(dhsdir, "contemprary.dta"))  # file name matches Stata (typo preserved)

# keep if GSMYEAR == DHSYEAR
contemp = contemp.loc[contemp["GSMYEAR"] == contemp["DHSYEAR"]].copy()

# Standardize lt_GSMCOVER_dist{bfd}
for bfd in [5,10,20,30,40,50,100]:
    var = f"lt_GSMCOVER_dist{bfd}"
    if var in contemp.columns:
        contemp[f"std_lt_GSMCOVER_dist{bfd}"] = zscore(contemp[var])

# Controls / outcomes
weather_l1 = ["wind_m_l1","Precipatation_s_l1","vapour_m_l1","solar_m_l1","temair_m_l1","rain_s_l1"]
individual_emp = ["age","highest_education","wealth_index"]
outcomes_emp = ["empl_current","empl_year","empl_cont","self_empl","work_wage"]

# Conditions and labels
conditions = [
    ("full","Full_Sample"),
    ("northern==0","Excluding_Northern"),
    ("place==0","Rural"),
    ("place==1","Urban"),
    ("age>=15 & age<=25","Age_15-25"),
    ("DHSYEAR==2013","DHSYEAR==2013"),
    ("DHSYEAR==2018","DHSYEAR==2018"),
]

# FE logic: if condition is a single year, FE=lga; else FE=lga DHSYEAR
def fe_logic_emp(cond):
    if "DHSYEAR==2013" in cond or "DHSYEAR==2018" in cond:
        return "lga"
    else:
        return "lga DHSYEAR"

res_emp = run_block(
    df=contemp,
    outcomes=outcomes_emp,
    bfd_list=[5,10,20,30],
    std_prefix="std_lt_GSMCOVER_dist{bfd}",
    raw_prefix="lt_GSMCOVER_dist{bfd}",
    file_suffix="Employment_LGA.xlsx",
    title_prefix="Impact of Mobile Internet Coverage on Employment - {label}",
    controls_weather=weather_l1,
    controls_individual=individual_emp,
    condition_pairs=conditions,
    fixed_effect_logic=fe_logic_emp
)
write_results_dict_to_workbook(res_emp, os.path.join(finaldir, "Employment_LGA.xlsx"))

In [ ]:
# ============================================================
# Section 2: Employment by occupation groups
# ============================================================
outcomes_occ = ["occ_traditional","occ_transitional","occ_mixed","occ_modern"]

res_occ = run_block(
    df=contemp,
    outcomes=outcomes_occ,
    bfd_list=[5,10,20,30],
    std_prefix="std_lt_GSMCOVER_dist{bfd}",
    raw_prefix="lt_GSMCOVER_dist{bfd}",
    file_suffix="Employment_LGA_Different Occupation.xlsx",
    title_prefix="Impact of Mobile Internet Coverage on Employment - {label}",
    controls_weather=weather_l1,
    controls_individual=individual_emp,
    condition_pairs=conditions,
    fixed_effect_logic=fe_logic_emp
)
write_results_dict_to_workbook(res_occ, os.path.join(finaldir, "Employment_LGA_Different Occupation.xlsx"))


In [ ]:
# ============================================================
# Section 3: Employment by skill levels
# ============================================================
outcomes_skill = ["highskill","modskill","unskill"]

res_skill = run_block(
    df=contemp,
    outcomes=outcomes_skill,
    bfd_list=[5,10,20,30],
    std_prefix="std_lt_GSMCOVER_dist{bfd}",
    raw_prefix="lt_GSMCOVER_dist{bfd}",
    file_suffix="Employment_LGA_Different_Skills.xlsx",
    title_prefix="Impact of Mobile Internet Coverage on Skills - {label}",
    controls_weather=weather_l1,
    controls_individual=individual_emp,
    condition_pairs=conditions,
    fixed_effect_logic=fe_logic_emp
)
write_results_dict_to_workbook(res_skill, os.path.join(finaldir, "Employment_LGA_Different_Skills.xlsx"))


In [ ]:
# ============================================================
# Section 4: Long-run effect — baseline from 2012
# ============================================================
lr12 = pd.read_stata(os.path.join(dhsdir, "contemprary.dta"))

# For each bfd, create GSMCOVER_dist{bfd}_12 = max value for 2012 by caseid
for bfd in [5,10,20,30,40,50,100]:
    base = f"GSMCOVER_dist{bfd}"
    tmp = lr12.loc[lr12["GSMYEAR"]==2012, ["caseid", base]].rename(columns={base: f"{base}_12_tmp"})
    lr12 = lr12.merge(tmp.groupby("caseid").max(), on="caseid", how="left")
    lr12.rename(columns={f"{base}_12_tmp": f"{base}_12"}, inplace=True)
    lr12[f"std_{base}_12"] = zscore(lr12[f"{base}_12"])

# keep if GSMYEAR == DHSYEAR
lr12 = lr12.loc[lr12["GSMYEAR"] == lr12["DHSYEAR"]].copy()

# FE logic same as before
def fe_logic_lr(cond):
    if "DHSYEAR==2013" in cond or "DHSYEAR==2018" in cond:
        return "lga"
    else:
        return "lga DHSYEAR"

res_lr12 = run_block(
    df=lr12,
    outcomes=outcomes_emp,
    bfd_list=[5,10,20,30],
    std_prefix="std_GSMCOVER_dist{bfd}_12",
    raw_prefix="GSMCOVER_dist{bfd}_12",
    file_suffix="Employment_LGA_Longrun.xlsx",
    title_prefix="Impact of Mobile Internet Coverage on Employment - {label}",
    controls_weather=weather_l1,
    controls_individual=individual_emp,
    condition_pairs=conditions,
    fixed_effect_logic=fe_logic_lr
)
write_results_dict_to_workbook(res_lr12, os.path.join(finaldir, "Employment_LGA_Longrun.xlsx"))


In [ ]:
# ============================================================
# Section 5: Long-run effect — average of 2012 & 2013
# ============================================================
lr1213 = pd.read_stata(os.path.join(dhsdir, "contemprary.dta"))

# GSMCOVER_dist{bfd}_1213 = mean over GSMYEAR in {2012,2013} by caseid
for bfd in [5,10,20,30,40,50,100]:
    base = f"GSMCOVER_dist{bfd}"
    tmp = lr1213.loc[lr1213["GSMYEAR"].isin([2012,2013]), ["caseid", base]]
    tmp = tmp.groupby("caseid")[base].mean().rename(f"{base}_1213").reset_index()
    lr1213 = lr1213.merge(tmp, on="caseid", how="left")
    lr1213[f"std_{base}_1213"] = zscore(lr1213[f"{base}_1213"])

# keep if GSMYEAR == DHSYEAR
lr1213 = lr1213.loc[lr1213["GSMYEAR"] == lr1213["DHSYEAR"]].copy()

res_lr1213 = run_block(
    df=lr1213,
    outcomes=outcomes_emp,
    bfd_list=[5,10,20,30],
    std_prefix="std_GSMCOVER_dist{bfd}_1213",
    raw_prefix="GSMCOVER_dist{bfd}_1213",
    file_suffix="Employment_LGA_Longrun_1213.xlsx",
    title_prefix="Impact of Mobile Internet Coverage on Employment - {label}",
    controls_weather=weather_l1,
    controls_individual=individual_emp,
    condition_pairs=conditions,
    fixed_effect_logic=fe_logic_lr
)
write_results_dict_to_workbook(res_lr1213, os.path.join(finaldir, "Employment_LGA_Longrun_1213.xlsx"))


In [ ]:
# ============================================================
# Section 6: Effect on husband occupation (men)
# ============================================================
pre = pd.read_stata(os.path.join(dhsdir, "preanalysis.dta"))  # source for v701, v705, v730, etc.
pre = pre[["caseid","v701","v705","v730"]].copy()

# recodes
pre["v705"] = pre["v705"].replace({96: np.nan, 98: np.nan, 99: np.nan})
# 1=working, 0=otherwise
empl_current_men = pre["v705"].isin([1,2,3,4,5,6,7,8,9]).astype(float)  # any listed category -> working
pre["empl_current_men"] = empl_current_men

# Occupation sector (v705 categories per your mapping)
occ_sector_men = pd.Series(np.nan, index=pre.index)

# 1 Traditional (agriculture: 4,5)
occ_sector_men[(pre["v705"]==4) | (pre["v705"]==5)] = 1
# 2 Transitional (household/domestic/services: 6,7)
occ_sector_men[(pre["v705"]==6) | (pre["v705"]==7)] = 2
# 3 Mixed (sales/manual: 3,8,9)
occ_sector_men[(pre["v705"]==3) | (pre["v705"]==8) | (pre["v705"]==9)] = 3
# 4 Modern (professional/clerical: 1,2)
occ_sector_men[(pre["v705"]==1) | (pre["v705"]==2)] = 4

pre["occ_sector_men"] = occ_sector_men

# Dummy groups
pre["occ_traditional_men"]  = (pre["occ_sector_men"]==1).astype(float)
pre["occ_transitional_men"] = (pre["occ_sector_men"]==2).astype(float)
pre["occ_mixed_men"]        = (pre["occ_sector_men"]==3).astype(float)
pre["occ_modern_men"]       = (pre["occ_sector_men"]==4).astype(float)

# Wage work dummy per your mapping: wage=1 if 2,3,5,7,8,9; else 0 if 4,6
pre["work_wage_men"] = pre["v705"].isin([2,3,5,7,8,9]).astype(float)
pre.loc[pre["v705"].isin([4,6]), "work_wage_men"] = 0.0

# Skill groups
pre["highskill_men"] = (pre["v705"]==1).astype(float)
pre["modskill_men"]  = pre["v705"].isin([2,3,5,7,8]).astype(float)
pre["unskill_men"]   = pre["v705"].isin([4,6,9]).astype(float)

# Education and age
pre["v701"] = pre["v701"].replace({8: np.nan, 9: np.nan})
pre.rename(columns={"v701":"highest_education_men"}, inplace=True)
pre.loc[(pre["v730"]>85) & (~pre["v730"].isna()), "v730"] = 85
pre.rename(columns={"v730":"age_men"}, inplace=True)

In [ ]:
# Save temp (optional)
pre.to_stata(os.path.join(tempdir, "temp.dta"), write_index=False)

# Merge with contemporaneous (standardize lt variable)
men = pd.read_stata(os.path.join(dhsdir, "contemprary.dta"))
for bfd in [5,10,20,30,40,50,100]:
    var = f"lt_GSMCOVER_dist{bfd}"
    if var in men.columns:
        men[f"std_lt_GSMCOVER_dist{bfd}"] = zscore(men[var])

men = men.loc[men["GSMYEAR"] == men["DHSYEAR"]].copy()
men = men.merge(pre, on="caseid", how="inner")

# Men outcomes and controls
weather_men = weather_l1
individual_men = ["age_men","highest_education_men","wealth_index"]
outcomes_men = ["empl_current_men","work_wage_men","highskill_men","modskill_men","unskill_men"]

# Conditions (same)
def fe_logic_men(cond):
    if "DHSYEAR==2013" in cond or "DHSYEAR==2018" in cond:
        return "lga"
    else:
        return "lga DHSYEAR"

# Only bfd=20 (per your Stata)
res_men = run_block(
    df=men,
    outcomes=outcomes_men,
    bfd_list=[20],
    std_prefix="std_lt_GSMCOVER_dist{bfd}",
    raw_prefix="lt_GSMCOVER_dist{bfd}",
    file_suffix="Employment_LGA_men.xlsx",
    title_prefix="Impact of Mobile Internet Coverage on Husband' Employment - {label}",
    controls_weather=weather_men,
    controls_individual=individual_men,
    condition_pairs=conditions,
    fixed_effect_logic=fe_logic_men
)
write_results_dict_to_workbook(res_men, os.path.join(finaldir, "Employment_LGA_men.xlsx"))

print("Done. Workbooks saved in:", finaldir)